决策树（decision tree）算法可分别应用于分类与回归。其学习过程分为：**特征选择、决策树生成、决策树的修剪，**主要算法包括ID3、C4.5、CART

# 一、特征选择

特征选择就是选择对于样本数据具有分类能力的特征，从而提高分类决策的效率。如果利用某一个特征分类的效果与随机分类效果无疑，则可称此特征无分类能力，这样的特征舍弃也无妨。
特征的选择准则是通过计算**信息增益**
## 一、信息增益
给定数据集D：<br>
其样本数量为|D|，即样本容量；<br>
设数据集有K个分类$C_k$，k=1,2,...，K；<br>
$|C_k|$表示属于分类$|C_k|$的样本个数；<br>
$\sum_{k=1}^{K}{|C_k|}=|D|$；<br>
根据特征A可将D分类n个子集，分别为$D_1, D_2, ..., D_n$；<br>
$|D_i|$为$D_i$的样本个数，$\sum_{i=1}^{n}{|D_i|}=|D|$；<br>
集合$D_{ik}$表示子集$D_i$中属于类$C_k$的样本，即$D_{ik} = D_i \cap C_k$；<br>
$|D_{ik}|$表示$D_{ik}$的个数。<br>
**信息增益算法：**<br>
输入：训练数据集D和特征A；<br>
输出：特征A对训练数据集D的信息增益Gain（D，A）<br>
进一步，输出：使得信息增益Gain（D，A）最大的特征$A_v$。<br>

(1)计算数据集D的经验熵$E(D)$ (entropy)
$$
E(D) = -\sum_{k=1}^{K}{p_klog_2(p_k)} = -\sum_{k=1}^{K}{\frac{|D_k|}{|D|}log_2(\frac{|D_k|}{|D|})}
$$
(2)计算特征A对数据集D的经验条件熵$E(D|A)$
$$
H(D|A) = -\sum_{i=1}^{n}{p_iE(D_i)} = -\sum_{i=1}^{n}{\sum_{k=1}^{K}{p_iE(D_{ik})}} = -\sum_{i=1}^{n}{\frac{|D_{i}|}{|D|}\sum_{k=1}^{K}{\frac{|D_{ik}|}{|D_i|}log_2(\frac{|D_{ik}|}{|D_i|})}}
$$
(3)计算信息增益
$$
Gain(D,A) = H(D)-H(D|A)
$$



In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#数据生成
X = np.random.randint(4,size=(100,4))
y = np.random.randint(6,size=(100,1))
D = pd.DataFrame(np.column_stack([X,y]))
D.shape

In [ ]:
#经验熵E（D）计算

pk = D.groupby(by=D.shape[1]-1).count()[0].values/len(D)
ED = -np.dot(pk,np.log2(pk))
ED

In [ ]:
#经验条件熵E（D|A）_EDK 、信息增益GAIN 计算
A = list([])
for v in range(D.shape[1]-1):
    Av = D.groupby([v,D.shape[1]-1])[1].count().unstack().fillna(0)
    Av = Av.values.astype(float)
    
    Av_sum = Av.sum(1)
    EDK = 0
    for ll in range(len(Av_sum)):
        Av[ll] = Av[ll]* (1/(Av_sum[ll]))
        EDK += np.dot(Av[ll],np.log2(Av[ll]))
    EDK = -EDK
    print(ED -  EDK)


### 分析
数据中出现"nan"的原因是某个$|D_{ik}|=0$，造成$\frac{|D_{ik}|}{|D_i|}=0$,然后出现了$log_2(\frac{|D_{ik}|}{|D_i|})=log_2(0)$的计算.<br>
当某个特征i没有对应的分类k的样本就会出现$|D_{ik}|=0$的现象，具体说来，如西瓜书中的案例，所有西瓜进行分类，分为"好瓜"、"坏瓜"，我要通过特征"皮的厚度"进行再分类，该分类可分为"厚"、"薄"2类，然后发现皮"薄"且为"好瓜"的有很多个，但是皮"厚"且为"好瓜"的样本数为0，所以，当i=("皮的厚度"，"厚")，k=（"好瓜"）时，就会出现$log_2(\frac{|D_{ik}|}{|D_i|})=log_2(0)$的计算。<br>
在本计算中，可根据实际的意义，令$log2(0) = 0$

In [ ]:
A = list([])
for v in range(D.shape[1]-1):
    Av = D.groupby([v,D.shape[1]-1])[1].count().unstack().fillna(0)
    Av = Av.values.astype(float)
    
    Av_sum = Av.sum(1)
    EDK = 0
    for ll in range(len(Av_sum)):
        Av[ll] = Av[ll]* (1/(Av_sum[ll]))
        tempAv = np.log2(Av[ll])
        tempAv[np.isinf(tempAv)] = 0
        EDK += np.dot(Av[ll],tempAv)
    EDK = -EDK
    print(ED -  EDK)

虽然还是会报错， 但是结果已经没有nan了

更换一组数据

In [166]:
#数据的读取
D = pd.read_csv('bank.csv',';')
D.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


### ID3算法
  **输入**：训练数据集D，特征集A，阈值$ \epsilon $ <br>
  **输出**：决策树T
 1. 若D中的所有实例属于$C_k$,则标记T节点为单节点，将类$C_k$作为该节点的类标记，并返回T；
 2. 若特征集A为空，则标记T节点为单节点，将类$C_k$作为该节点的类标记，并返回T；
 3. 否则，按照算法计算A中各特征的信息增益，并选出信息增益最大的$A_g$；
 4. 如果$A_g<\epsilon$，则置T为单节点树，并将实例数最大的类$C_k$作为该节点的类标记，返回T；
 5. 否则，对$A_g$的每个可能值$a_i$，依$A_g = a_i$将D划分为若干个非空子集$D_i$，将$D_i$中实例数最大的类作为标记，构建子节点，由节点及其子节点构建树T，返回T；
 6. 对第i个子节点，以$D_i$为训练集，以$A-A_g$为特征集，阈值$\epsilon$递归调用算法，得到子树$T_i$，返回$T_i$
  

In [164]:
class DecisionTree(object):
    def __init__(self):
        self.myTree = {}
        self.dep = 0
        self.n_div = 5
        self.maxkind = 100
    def Divn(data):
        '''
        划分一串以为数组
        输入：一维数组
        输出：划分后的一位数组
        '''
        return int((data - np.min(data))/(np.max(data)-np.min(data))*self.n_div)
    
    def DataDivide(self, inData):
        '''
        划分一个DataFrame
        输入：DataFrame
        输出：将DataFrame中column的数据种类大于标准的进行分类划分。
        '''
        dataLenth = inData.shape[0]
        for col in inData.columns:
            if len(np.unique(inData[col]))> self.maxkind and inData[col][0] is not str :
                inData[col] = self.Divn(inData[col])
        return inData
    
    def CreateTree(self,Data, thisTree,e):
        '''
        根据输入数据矩阵，计算获得决策树myTree
        其中决策树myTree为class的内部变量，为dict类型
        输入：
            Data：训练数据（在递归中变化）
            thisTree：树dict（在递归中变化）
            e ：   最大信息增益，用于控制递归的深度
        输出： 树dict
        '''
        n_dimensions = Data.shape[1]      #数据维度
        m_samples = Data.shape[0]         #样本数量
        nSample,samples = self.CountSamples(Data.iloc[:,n_dimensions - 1 ])
        maxclass = Data.iloc[:,-1].groupby(Data.iloc[:,-1]).count().sort_values(ascending = False).index[0]
        if nSample == 1: 
            print(samples[0])
            thisTree = {samples[0]}           
            return thisTree
        elif n_dimensions <= 1:
            thisTree = {maxclass}
            return     thisTree
        #否则计算最大信息增益
        maxGainName, maxGain = self.GainCal(Data.copy())
        if maxGain <= e :
            thisTree = {maxclass}
            return thisTree
        print(maxGain,end='\t')          
        thisTree = {maxGainName:{}}
        thisTree[maxGainName] = {i:0 for i in np.unique(Data[maxGainName])}
        maxcount = 0
        for i_feature in thisTree[maxGainName]:
            nD = Data[Data[maxGainName]==i_feature].drop([maxGainName],axis=1)            
            y = np.unique(nD.iloc[:,-1])            
            if len(y) == 1:
                thisTree[maxGainName][i_feature] = y
            else:
                thisTree[maxGainName][i_feature] = self.CreateTree(nD,thisTree[maxGainName][i_feature],e)                
        return thisTree
    
    def MaxClass(self, Data):
        '''
        计算Data中y的最大类别的分类
        '''
        return Data.iloc[:,-1].groupby(Data.iloc[:,-1]).count().sort_values(ascending = False).index[0]        
    
    def GainCal(self,inData):
        '''
        输入：矩阵数据，
            计算信息增益
        输出：
            maxGainIndex :  最大信息增益的特征index
            listGain[maxGainIndex]: 最大信息增益值
        '''
        n_dimensions = inData.shape[1]      #数据维度
        m_samples = inData.shape[0]         #样本数量
        col = inData.columns
        inData.columns = range(n_dimensions)
        pk = inData.groupby(by=n_dimensions-1).count()[0].values/m_samples
        ED = -np.dot(pk,np.log2(pk))
        A = list([])
        for v in range(inData.shape[1]-1):
            Av = inData.groupby([v,inData.iloc[:,-1]])[1].count().unstack().fillna(0).values.astype(float)
            Av_sum = Av.sum(1)
            EDK = 0
            for ll in range(len(Av_sum)):
                Av[ll] = Av[ll]/(Av_sum[ll])
                tempAv = np.log2(Av[ll])
                tempAv[np.isinf(tempAv)] = 0
                EDK += np.dot(Av[ll],tempAv)
            A.append([v,(ED -  EDK)])
        listGain = np.array(A)     
        maxGainIndex = int(listGain[listGain[:,1]==listGain[:,1].max()][0,0])        
        maxGain = listGain[:,1].max()
        return col[maxGainIndex],maxGain
    
    def CountSamples(self,inData):
        '''
        输入：单维度数列
        输出：以数据矩阵最后一列为结果，汇总结果种类
            nSample: 结果的数量
            samples : 具体包含那些结果
        '''
        samples = np.unique(inData)
        nSample = len(samples)
        return nSample,samples                            
    def fit(self,inData, e=4):
        inData = self.DataDivide(inData)
        self.myTree = self.CreateTree(self,inData, {} ,e)     
        
    def predict(self,X):
        '''
        根据训练数据生成树后，进行预测
        输入：测试数据
        输出：预测结果
        '''
        return self.Find(X,self.myTree)
        
    def Find(self,X,Tree):   
        '''
        根据测试数据、树结构，递归生成结果
        输入：测试数据，树结构
        输出：预测结果
        '''
        if type(Tree) is not dict:   #判断本树的类型，如果是dict则继续递归，不是dict必然就到最终节点了，则返回Tree
            return Tree
        for col in X.columns:
            if col in Tree:
                name,value = col,X[col].values[0]
        nextx = X.drop([r1],axis=1)
  
        return self.Find(nextx,Tree[name][value])
dt = DecisionTree()
dt.fit(D.copy())


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1.0,unemployed,married,primary,no,0.0,no,no,cellular,3.0,oct,0.0,0.0,0.0,0.0,unknown,no
1,1.0,services,married,secondary,no,1.0,yes,yes,cellular,2.0,may,0.0,0.0,2.0,1.0,failure,no
2,1.0,management,single,tertiary,no,0.0,yes,no,cellular,2.0,apr,0.0,0.0,2.0,0.0,failure,no
3,1.0,management,married,tertiary,no,0.0,yes,yes,unknown,0.0,jun,0.0,0.0,0.0,0.0,unknown,no
4,3.0,blue-collar,married,secondary,no,0.0,yes,no,unknown,1.0,may,0.0,0.0,0.0,0.0,unknown,no
